In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
train_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/train'
valid_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/valid'
test_dir = '/content/drive/MyDrive/Colab Notebooks/fotos_tiempo/test'

#Generadores
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Parametros
input_shape = (64, 64, 3)
num_classes = 11



batch_size = [4,8,12]
epochs=[6]#4,6,8
optimizador=["adam","SGD","adagrad"]
capas=[3,4,2]
dfError = pd.DataFrame(columns=["epocas", "batch","optimizador","capas","accuracy"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for epochs_ind in range(len(epochs)):
  for batch_ind in range(len(batch_size)):
    for opt_ind in range(len(optimizador)):
      for capas_ind in range(len(capas)):


          train_generator = train_datagen.flow_from_directory(
              train_dir,
              target_size=(input_shape[0], input_shape[1]),
              batch_size=batch_size[batch_ind],
              class_mode='categorical')

          test_generator = test_datagen.flow_from_directory(
              test_dir,
              target_size=(input_shape[0], input_shape[1]),
              batch_size=batch_size[batch_ind],
              class_mode='categorical')

          validation_generator = validation_datagen.flow_from_directory(
              valid_dir,
              target_size=(input_shape[0], input_shape[1]),
              batch_size=batch_size[batch_ind],
              class_mode='categorical')



          # Modelo
          if(capas[capas_ind]==3):
            model = models.Sequential([
                layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(64, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(128, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Flatten(),
                layers.Dense(128, activation='relu'),
                layers.Dense(num_classes, activation='softmax')
            ])
          elif(capas[capas_ind]==4):
            model = models.Sequential([
                layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(64, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(128, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(256, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Flatten(),
                layers.Dense(128, activation='relu'),
                layers.Dense(num_classes, activation='softmax')
            ])

          else:
            model = models.Sequential([
                layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(64, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Flatten(),
                layers.Dense(128, activation='relu'),
                layers.Dense(num_classes, activation='softmax')
            ])


          # Compilar el modelo
          model.compile(optimizer=optimizador[opt_ind],
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])



          # Entrenar el modelo
          history = model.fit(
              train_generator,
              steps_per_epoch=train_generator.samples // batch_size[batch_ind],
              epochs=epochs[epochs_ind],
              validation_data=validation_generator,
              validation_steps=validation_generator.samples // batch_size[batch_ind])



          # Evaluar precision del modelo con el conjunto de prueba
          test_loss, test_acc = model.evaluate(test_generator, verbose=2)
          print('Test accuracy:', test_acc)
          print("epocas:",str(epochs[epochs_ind]))
          print("batch_size:",str(batch_size[batch_ind]))
          print("optimizador:",str(optimizador[opt_ind]))
          print("capas:",str(capas[capas_ind]))
          dfError.loc[len(dfError)] = [str(epochs[epochs_ind]),str(batch_size[batch_ind]),str(optimizador[opt_ind]), str(capas[capas_ind]), test_acc]
          print(dfError)

Found 253 images belonging to 11 classes.
Found 132 images belonging to 11 classes.
Found 132 images belonging to 11 classes.
Epoch 1/6
63/63 [==============================] - 9s 120ms/step - loss: 2.4129 - accuracy: 0.0723 - val_loss: 2.3228 - val_accuracy: 0.1818
Epoch 2/6
63/63 [==============================] - 5s 78ms/step - loss: 2.2911 - accuracy: 0.1847 - val_loss: 2.2193 - val_accuracy: 0.2424
Epoch 3/6
63/63 [==============================] - 7s 105ms/step - loss: 2.0697 - accuracy: 0.2811 - val_loss: 1.8904 - val_accuracy: 0.3485
Epoch 4/6
63/63 [==============================] - 5s 77ms/step - loss: 1.9082 - accuracy: 0.3574 - val_loss: 1.9677 - val_accuracy: 0.3864
Epoch 5/6
63/63 [==============================] - 7s 104ms/step - loss: 1.7050 - accuracy: 0.4016 - val_loss: 1.7867 - val_accuracy: 0.3788
Epoch 6/6
63/63 [==============================] - 5s 79ms/step - loss: 1.5365 - accuracy: 0.4739 - val_loss: 1.6067 - val_accuracy: 0.4318
33/33 - 1s - loss: 1.7885 - acc

In [ ]:
dfError.to_csv('/content/drive/MyDrive/Colab Notebooks/erroresTensorKeras.csv', index=False)